### Import libraries

Import libraries, set parameters, and import classes

In [ ]:
import numpy as np
import tensorflow as tf
import pickle
np.random.seed(1)
tf.random.set_seed(1)
tf.keras.backend.set_floatx('float32')

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (3,3)
plt.rcParams.update({'font.size': 13})

In [ ]:
from __lib__.TANN_class import TANN
from __lib__.DataProcessing import PreProcessing

### Import datasets and pre-processing

In [ ]:
file = './data/3D_PlaneStrain_VonMises_CSMA'
training_percentage = 0.6

In [ ]:
with open(file, 'rb') as f_obj:
    data = pickle.load(f_obj)

strain,strainDot,svars,svarsDot,stress,energy,dissipationDot = data

dissipationDot = dissipationDot[:,0]
del data

##### Split datasets  (tv=training+validation - test)

In [ ]:
PreProcessing = PreProcessing(0.6)

train_p,val_p,test_p = PreProcessing.split_dataset(strain,percentage=True)

print('# samples:', train_p+val_p+test_p,
      '\n# training:', train_p,
      '\n# validation:', val_p,
      '\n# test:', test_p)

# samples: 21000 
# training: 12600 
# validation: 4200 
# test: 4200


In [ ]:
strain_tv,strain_test = PreProcessing.split_dataset(strain)
strainDot_tv,strainDot_test = PreProcessing.split_dataset(strainDot)

stress_tv,stress_test = PreProcessing.split_dataset(stress)

svars_tv,svars_test = PreProcessing.split_dataset(svars)
svarsDot_tv,svarsDot_test = PreProcessing.split_dataset(svarsDot)

energy_tv,energy_test = PreProcessing.split_dataset(energy)
dissipationDot_tv,dissipationDot_test = PreProcessing.split_dataset(dissipationDot)

##### Get normalization parameters

In [ ]:
# Use simply the code you programmed previously for the elastic case

In [ ]:
#Or:

# prm_E = PreProcessing.GetParams(strain_tv,normalize=True)
# prm_EDot = PreProcessing.GetParams(strainDot_tv,normalize=True)
# prm_S = PreProcessing.GetParams(stress_tv,normalize=True)

# prm_F = PreProcessing.GetParams(energy_tv,normalize_01=True)
# prm_D = PreProcessing.GetParams(dissipationDot_tv,normalize_01=True)

# prm_Z = PreProcessing.GetParams(svars_tv,normalize=True)
# prm_ZDot = PreProcessing.GetParams(svarsDot_tv,normalize=True)

##### Normalize datasets

In [ ]:
# Training and validation datasets

__strain_tv,__strain_test = PreProcessing.Normalize(
    [strain_tv,strain_test],prm_E,train_val=True)
__strainDot_tv,__strainDot_test = PreProcessing.Normalize(
    [strainDot_tv,strainDot_test],prm_EDot,train_val=True)

__svars_tv,__svars_test = PreProcessing.Normalize(
    [svars_tv,svars_test],prm_Z,train_val=True)
__svarsDot_tv,__svarsDot_test = PreProcessing.Normalize(
    [svarsDot_tv,svarsDot_test],prm_ZDot,train_val=True)

__stress_tv,__stress_test = PreProcessing.Normalize(
    [stress_tv,stress_test],prm_S,train_val=True)
__energy_tv,__energy_test = PreProcessing.Normalize(
    [energy_tv,energy_test],prm_F,train_val=True)
__dissipationDot_tv,__dissipationDot_test = PreProcessing.Normalize(
    [dissipationDot_tv,dissipationDot_test],prm_D,train_val=True)

### Thermodynamics-based Artificial Neural Networks

#### Build network

In [ ]:
umat_dim = 4 # dimensional space

hidden_NN_Energy = 4 # number hidden layers in NN Energy
hidden_NN_Evolution = 3 # number of hidden layers in NN Evolution
material = 'inelastic'

TANNnet = TANN(umat_dim,
               material,
               hidden_NN_Energy,
               activation_NN_Energy='fast_gelu',
               hidden_NN_Evolution = hidden_NN_Evolution,
               activation_NN_Evolution = 'leaky_relu') # Initialize TANN class

params = [prm_E,prm_S,prm_Z,prm_F,prm_D,prm_EDot,prm_ZDot] # regroup normalization parameters
TANNnet.SetParams(params) # set normalization parameters in the model

# build TF model class
input_shape = [[None,umat_dim],[None,umat_dim],[None,umat_dim]]
TANNnet.build(input_shape)

# print model's summary
print(TANNnet.summary())

Model: "Thermodynamics-based_Artificial_Neural_Networks"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NN_Energy (Sequential)      (None, 1)                 977       
                                                                 
 NN_Evolution (Sequential)   (None, 4)                 20260     
                                                                 
Total params: 21,237
Trainable params: 21,237
Non-trainable params: 0
_________________________________________________________________
None


#### Define inputs and outputs

In [ ]:
output_tv = [__svarsDot_tv,__energy_tv,__stress_tv,__dissipationDot_tv]

output_test = [__svarsDot_test,__energy_test,__stress_test,__dissipationDot_test]

input_tv = [__svars_tv,__strain_tv,__strainDot_tv]

input_test = [__svars_test,__strain_test,__strainDot_test]

NameError: name '__svarsDot_tv' is not defined

#### Compile and train the network

In [ ]:
TANNnet.compileTANN()

callbacks_list = [tf.keras.callbacks.EarlyStopping(monitor ='val_loss',min_delta = 1.e-6,
                                                   patience = 200, verbose = 0, mode = 'min',
                                                   baseline = None,restore_best_weights = True)]

hist_TANNnet = TANNnet.fit(input_tv,output_tv,
                           epochs = 1500,
                           validation_split=0.2,
                           batch_size = 200,
                           verbose = 2,
                           use_multiprocessing=True,
                           callbacks = callbacks_list,)


NameError: name 'input_tv' is not defined

In [ ]:
print('Evaluate loss for training and validation datasets:'); TANNnet.evaluate(input_tv,output_tv,verbose=2)
print('\nEvaluate loss for test datasets:'); TANNnet.evaluate(input_test,output_test,verbose=2)


svarsDot_loss = np.asarray(hist_TANNnet.history['output_1_loss'])
svarsDot_val_loss = np.asarray(hist_TANNnet.history['val_output_1_loss'])
energy_loss = np.asarray(hist_TANNnet.history['output_2_loss'])
energy_val_loss = np.asarray(hist_TANNnet.history['val_output_2_loss'])
stress_loss = np.asarray(hist_TANNnet.history['output_3_loss'])
stress_val_loss = np.asarray(hist_TANNnet.history['val_output_3_loss'])
dissipationDot_loss = np.asarray(hist_TANNnet.history['output_4_loss'])
dissipationDot_val_loss = np.asarray(hist_TANNnet.history['val_output_4_loss'])
epochs = range(1, len(energy_loss) + 1)

In [ ]:
fig, ax = plt.subplots(1)
ax.loglog(epochs, svarsDot_loss, color='red', label='$\dot{z}$', linewidth=2, marker='o',markersize=0)
ax.loglog(epochs, energy_loss, color='darkblue', label='$\psi$', linewidth=2, marker='o',markersize=0)
ax.loglog(epochs, stress_loss, color='black', label='$\psi$', linewidth=2, marker='o',markersize=0)
ax.loglog(epochs, dissipationDot_loss, color='darkorange', label='$d$', linewidth=2, marker='o',markersize=0)

ax.loglog(epochs, svarsDot_val_loss, color='red', alpha=0.5,linewidth=2, marker='o',markersize=0)
ax.loglog(epochs, energy_val_loss, color='darkblue', alpha=0.5,  linewidth=2, marker='o',markersize=0)
ax.loglog(epochs, stress_val_loss, color='black', alpha=0.5,linewidth=2, marker='o',markersize=0)
ax.loglog(epochs, dissipationDot_val_loss, color='darkorange', alpha=0.5,  linewidth=2, marker='o',markersize=0)


ax.set_xlabel('epochs')
ax.set_ylabel('Mean Absolute Error')
ax.legend(loc='best')
ax.grid()
ax.set_ylim(1e-7,1)

In [ ]:
pred__svarsDot,pred__energy,pred__stress,pred__dissipationDot,pred_2ndLaw=TANNnet.predict(input_test)

# Plot stress
plt.plot(__stress_test,
         pred__stress,
         alpha=0.3,linewidth=0, marker='o',markersize=5)
plt.xlabel("$\sigma$")
plt.ylabel("$\sigma=\partial_{\\varepsilon} NN_{\psi}(\\varepsilon)$")
plt.show()

# Plot internal variable
plt.plot(__svarsDot_test,
         pred__svarsDot,
         alpha=0.3,linewidth=0, marker='o',markersize=5)
plt.xlabel("$\\dot{z}$")
plt.ylabel("$NN_{\\dot{z}}$")
plt.show()

# Plot energy
plt.plot(__energy_test,
         pred__energy,
         alpha=0.3,linewidth=0, marker='o',markersize=5)
plt.xlabel("$\psi$")
plt.ylabel("$\psi=NN_{\psi}(\\varepsilon,z)$")
plt.show()

# Plot dissipation rate
plt.plot(__dissipationDot_test,
         pred__dissipationDot,
         alpha=0.3,linewidth=0, marker='o',markersize=5)
plt.xlabel("$d$")
plt.ylabel("$d=-\partial_{z}NN_{\psi}\cdot \dot{z}$")
plt.show()

### Save model

In [ ]:
# filename = './results/TANN_VonMises'
# TANNnet.save_export(filename)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b33f54f1-f98d-4fed-99c0-2484f115edd9' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>